In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import mplhep as hep
hep.style.use("CMS")
from coffea import util
import itertools
import os, sys
import glob
import copy
import uproot

import mplhep as hep
hep.style.use("CMS")

sys.path.append('../python/')
import functions

In [3]:
IOV = '2018'

In [4]:
qcd16 = util.load(f'../outputs/scale/QCD_2016all.coffea')
qcd17 = util.load(f'../outputs/scale/QCD_2017.coffea')
qcd18 = util.load(f'../outputs/scale/QCD_2018.coffea')


ttbar17 = util.load(f'../outputs/scale/TTbar_2017.coffea')
ttbar16 = util.load(f'../outputs/scale/TTbar_2016all.coffea')
ttbar18 = util.load(f'../outputs/scale/TTbar_2018.coffea')


data17 = util.load(f'../outputs/scale/JetHT_2017.coffea')
data16 = util.load(f'../outputs/scale/JetHT_2016all.coffea')
data18 = util.load(f'../outputs/scale/JetHT_2018.coffea')


# qcd17 = util.load(f'../outputs/scale/QCD_2017.coffea')
# ttbar17 = util.load(f'../outputs/scale/TTbar_2017.coffea')
# data17 = util.load(f'../outputs/scale/JetHT_2017.coffea')

# qcd16 = util.load(f'../outputs/scale/QCD_2016all.coffea')
# ttbar16 = util.load(f'../outputs/scale/TTbar_2016all.coffea')
# data16 = util.load(f'../outputs/scale/JetHT_2016all.coffea')

## Cutflow

In [5]:
data = {}
cuts = []

for cut in data16['cutflow'].keys():
    if 'sumw' in cut: continue
        
    evts = data18['cutflow'][cut] + data17['cutflow'][cut] + data16['cutflow'][cut]
    data[cut] = evts
    cuts.append(evts)

        

In [6]:
df = pd.DataFrame(data=cuts, index=data.keys())
df

,0
all events,25390948
trigger,1602128
htCut,646226
metfilter,642122
jetid,642115
jetkincut,635899
twoFatJets,564812
oneTTbar,564812
dPhiCut,558948
Good Subjets,545105


In [7]:
table1 = '''
\\begin{table}
    \\begin{center}
        \\rowcolors{2}{gray!15}{white}
        \\begin{tabular}{ |p{6cm}||p{3cm}|  } \\hline 
            \\textbf{Cut} & \\textbf{Events} \\\\ \\hline
            All Events                                  &  '''+'{0:0,.0f}'.format(data['all events'])+''' \\\\
            Trigger                                     &  '''+'{0:0,.0f}'.format(data['trigger'])+''' \\\\ 
            $HT > 1400$                                 &   '''+'{0:0,.0f}'.format(data['htCut'])+''' \\\\ 
            METfilter                                   &   '''+'{0:0,.0f}'.format(data['metfilter'])+''' \\\\ 
            $\\ge 1$ AK8 jet passing JetID               &   '''+'{0:0,.0f}'.format(data['jetid'])+''' \\\\ 
            $\\ge 1$ AK8 $p_T > 400$ GeV, $|y| < 2.4$    &   '''+'{0:0,.0f}'.format(data['jetkincut'])+''' \\\\ 
            $>2$ AK8 Jets                               &   '''+'{0:0,.0f}'.format(data['twoFatJets'])+''' \\\\ 
            %$\\geq 1$ ttbar candidate                   &   '''+'{0:0,.0f}'.format(data['oneTTbar'])+''' \\\\
            $|\\Delta\\varphi| > 2.1$                     &   '''+'{0:0,.0f}'.format(data['dPhiCut'])+''' \\\\ 
            \\hline
        \\end{tabular}
        \\caption{Cutflow for blinded JetHT events for 2016, 2017 and 2018 datasets.}
        \\label{tab:cutflow}
    \\end{center}
\\end{table}

'''

In [8]:
print(table1)


\begin{table}
    \begin{center}
        \rowcolors{2}{gray!15}{white}
        \begin{tabular}{ |p{6cm}||p{3cm}|  } \hline 
            \textbf{Cut} & \textbf{Events} \\ \hline
            All Events                                  &  25,390,948 \\
            Trigger                                     &  1,602,128 \\ 
            $HT > 1400$                                 &   646,226 \\ 
            METfilter                                   &   642,122 \\ 
            $\ge 1$ AK8 jet passing JetID               &   642,115 \\ 
            $\ge 1$ AK8 $p_T > 400$ GeV, $|y| < 2.4$    &   635,899 \\ 
            $>2$ AK8 Jets                               &   564,812 \\ 
            %$\geq 1$ ttbar candidate                   &   564,812 \\
            $|\Delta\varphi| > 2.1$                     &   558,948 \\ 
            \hline
        \end{tabular}
        \caption{Cutflow for blinded JetHT events for 2016, 2017 and 2018 datasets.}
        \label{tab:cutflow}
    \end{center}
\e

## Weights

In [9]:
wgts = {}
wgts['2018'] = ttbar18['weights']
wgts['2017'] = ttbar17['weights']
wgts['2016'] = ttbar16['weights']

wdata = {}
wdata2 = {}

for year in ['2016', '2017', '2018']:
    
    wdata[year] = {}
    wdata2[year] = {}
    nom = wgts[year]['nominal']
    
    for wgt in wgts[year]:
        
#         print(wgt)
        
        
        if not 'nominal' in wgt:
            
            try:

                wdata2[year][wgt] = wgts[year][wgt]/nom

                wdata[year][wgt] = np.abs(1-wgts[year][wgt]/nom) #np.abs(1-np.abs(wgts[wgt]/nom))
            except:
                continue


In [22]:
for year in wdata2.keys():
    print(year)
    
    for key in wdata2[year].keys():
    
        print(key, '{0:0.3f}'.format(wdata2[year][key]), '{0:0.3f}'.format(wdata[year][key]))

2016
pdfDown 1.017 0.017
q2Down 0.700 0.300
btagDown 1.000 0.000
prefiringUp 0.990 0.010
btagUp 1.000 0.000
prefiringDown 1.010 0.010
pileupDown 0.997 0.003
pileupUp 1.003 0.003
q2Up 1.468 0.468
pdfUp 1.009 0.009
jesUp 1.050 0.050
jesDown 0.571 0.429
jerUp 0.401 0.599
jerDown 0.199 0.801
2017
btagDown 1.000 0.000
pdfUp 1.009 0.009
btagUp 1.000 0.000
q2Up 1.466 0.466
q2Down 0.701 0.299
pileupDown 0.994 0.006
prefiringDown 1.010 0.010
prefiringUp 0.990 0.010
pdfDown 1.017 0.017
pileupUp 1.006 0.006
jesUp 1.049 0.049
jesDown 0.572 0.428
jerUp 0.402 0.598
jerDown 0.199 0.801
2018
pdfDown 1.183 0.183
q2Down 0.814 0.186
btagDown 1.162 0.162
btagUp 1.162 0.162
pileupUp 1.170 0.170
pdfUp 1.172 0.172
q2Up 1.705 0.705
pileupDown 1.155 0.155
jesUp 1.060 0.060
jesDown 0.629 0.371
jerUp 0.502 0.498
jerDown 0.332 0.668
hem 0.162 0.838


In [17]:
table2 = '''
\\begin{table}
    \\begin{center}
        \\begin{tabular}{ |p{1cm}||p{3cm}| |p{3cm}| } \\hline 
            \\textbf{Year} & \\textbf{Systematic} & \\textbf{up/down} \\\\ \\hline
              2016   & PileUp       &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2016']['pileupUp'], wdata['2016']['pileupDown'])+''' \\\\
                     & PDF          &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2016']['pdfUp'],    wdata['2016']['pdfDown'])+''' \\\\ 
                     & $Q^2$        &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2016']['q2Up'],     wdata['2016']['q2Down'])+''' \\\\ 
                     & btag         &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2016']['btagUp'],   wdata['2016']['btagDown'])+''' \\\\ 
                     & JES          &   '''+'-{0:0.2f}/-{1:0.2f}'.format(wdata['2016']['jesUp'],    wdata['2016']['jesDown'])+''' \\\\ 
                     & JER          &   '''+'-{0:0.2f}/-{1:0.2f}'.format(wdata['2016']['jerUp'],    wdata['2016']['jerDown'])+''' \\\\
                     & L1 Prefiring &   '''+'+{0:0.2f}/+{1:0.2f}'.format(wdata['2017']['prefiringUp'],    wdata['2017']['prefiringDown'])+''' \\\\
              \\hline    
              2017   & PileUp       &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2017']['pileupUp'], wdata['2017']['pileupDown'])+''' \\\\
                     & PDF          &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2017']['pdfUp'],    wdata['2017']['pdfDown'])+''' \\\\ 
                     & $Q^2$        &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2017']['q2Up'],     wdata['2017']['q2Down'])+''' \\\\ 
                     & btag         &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2017']['btagUp'],   wdata['2017']['btagDown'])+''' \\\\ 
                     & JES          &   '''+'-{0:0.2f}/-{1:0.2f}'.format(wdata['2017']['jesUp'],    wdata['2017']['jesDown'])+''' \\\\ 
                     & JER          &   '''+'-{0:0.2f}/-{1:0.2f}'.format(wdata['2017']['jerUp'],    wdata['2017']['jerDown'])+''' \\\\
                     & L1 Prefiring &   '''+'+{0:0.2f}/+{1:0.2f}'.format(wdata['2017']['prefiringUp'],    wdata['2017']['prefiringDown'])+''' \\\\
              \\hline
              2018   & PileUp       &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2018']['pileupUp'], wdata['2018']['pileupDown'])+''' \\\\
                     & PDF          &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2018']['pdfUp'],    wdata['2018']['pdfDown'])+''' \\\\ 
                     & $Q^2$        &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2018']['q2Up'],     wdata['2018']['q2Down'])+''' \\\\ 
                     & btag         &   '''+'+{0:0.2f}/-{1:0.2f}'.format(wdata['2018']['btagUp'],   wdata['2018']['btagDown'])+''' \\\\ 
                     & JES          &   '''+'-{0:0.2f}/-{1:0.2f}'.format(wdata['2018']['jesUp'],    wdata['2018']['jesDown'])+''' \\\\ 
                     & JER          &   '''+'-{0:0.2f}/-{1:0.2f}'.format(wdata['2018']['jerUp'],    wdata['2018']['jerDown'])+''' \\\\
                     & HEM          &   '''+'-{0:0.2f}'.format(wdata['2018']['hem'])+''' \\\\

               \\hline
        \\end{tabular}
        \\label{tab:syst}
        \\caption{Systematic uncertainties for 2016, 2017, 2018 MC TTbar datasets}
    \\end{center}
\\end{table}

'''

In [18]:
print(table2)


\begin{table}
    \begin{center}
        \begin{tabular}{ |p{1cm}||p{3cm}| |p{3cm}| } \hline 
            \textbf{Year} & \textbf{Systematic} & \textbf{up/down} \\ \hline
              2016   & PileUp       &   +0.00/-0.00 \\
                     & PDF          &   +0.01/-0.02 \\ 
                     & $Q^2$        &   +0.47/-0.30 \\ 
                     & btag         &   +0.00/-0.00 \\ 
                     & JES          &   -0.05/-0.43 \\ 
                     & JER          &   -0.60/-0.80 \\
                     & L1 Prefiring &   +0.01/+0.01 \\
              \hline    
              2017   & PileUp       &   +0.01/-0.01 \\
                     & PDF          &   +0.01/-0.02 \\ 
                     & $Q^2$        &   +0.47/-0.30 \\ 
                     & btag         &   +0.00/-0.00 \\ 
                     & JES          &   -0.05/-0.43 \\ 
                     & JER          &   -0.60/-0.80 \\
                     & L1 Prefiring &   +0.01/+0.01 \\
              \hline
    

In [ ]:
'+{0:0.2f}/-{1:0.2f}'.format(wdata['pileupUp'],wdata['pileupDown'])